# Community detection for Heterogeneous graph

## 0. Enviroment setup

In [1]:
# !pip uninstall torch torchvision torchaudio --yes
# !pip install torch==2.2.1 torchvision==0.17.1 torchaudio==2.2.1 --index-url https://download.pytorch.org/whl/cu121
# !pip install lightning torch_geometric
# !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.2.0+cu121.html
# !pip install wandb
# !pip install fast-pytorch-kmeans
# !pip install dbscan

In [2]:
import os
import shutil
import wandb
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

from torch_geometric.utils import negative_sampling
import torch_geometric.transforms as T
from torch_geometric.utils import train_test_split_edges
# from torch_geometric.loader import LinkNeighborLoader
# from torch_geometric.data.lightning import LightningLinkData

import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import WandbLogger

In [3]:
from network_analysis.models.community_detection import *
from preprocess import *

In [4]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [5]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## 1. Data Pre-processing

In [6]:
DATA_VERSION = 'final_v1'

In [7]:
# data = hive_preprocessing(
#     f'dataset/hive/{DATA_VERSION}/nodes_labelled.csv',
#     f'dataset/hive/{DATA_VERSION}/edges_labelled.csv',
#     to_undirected = True,
# )
# torch.save(data, f'dataset/hive/{DATA_VERSION}/hive.pt')
data = torch.load(f'dataset/hive/{DATA_VERSION}/hive.pt')
data

HeteroData(
  user={
    x=[18645, 5],
    y=[18645],
  },
  comment={
    x=[125111, 1],
    y=[125111],
  },
  post={
    x=[13540, 1],
    y=[13540],
  },
  (user, upvote, comment)={
    edge_index=[2, 423638],
    edge_attr=[423638, 1],
    y=[423638],
  },
  (user, upvote, post)={
    edge_index=[2, 554131],
    edge_attr=[554131, 1],
    y=[554131],
  },
  (user, write, comment)={
    edge_index=[2, 78696],
    edge_attr=[78696, 1],
    y=[78696],
  },
  (user, write, post)={
    edge_index=[2, 12958],
    edge_attr=[12958, 1],
    y=[12958],
  },
  (user, downvote, comment)={
    edge_index=[2, 6819],
    edge_attr=[6819, 1],
    y=[6819],
  },
  (user, downvote, post)={
    edge_index=[2, 2934],
    edge_attr=[2934, 1],
    y=[2934],
  },
  (comment, belong_to, comment)={
    edge_index=[2, 58911],
    edge_attr=[58911, 1],
    y=[58911],
  },
  (comment, belong_to, post)={
    edge_index=[2, 19838],
    edge_attr=[19838, 1],
    y=[19838],
  },
  (comment, rev_upvote, user)={


In [8]:
num_edges = len(data.edge_types)
edge_types = data.edge_types[:num_edges//2]

In [9]:
edge_types

[('user', 'upvote', 'comment'),
 ('user', 'upvote', 'post'),
 ('user', 'write', 'comment'),
 ('user', 'write', 'post'),
 ('user', 'downvote', 'comment'),
 ('user', 'downvote', 'post'),
 ('comment', 'belong_to', 'comment'),
 ('comment', 'belong_to', 'post')]

In [10]:
rev_edge_types = data.edge_types[num_edges//2:]

In [11]:
transform = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    disjoint_train_ratio=0.3, 
    add_negative_train_samples=True,
    neg_sampling_ratio=2.0,
    edge_types=edge_types,
    rev_edge_types=rev_edge_types, 
)

train_data, val_data, test_data = transform(data)

## 2. Training

In [12]:
model = GATv2(
    in_channels=-1, 
    out_channels= 16, 
    hidden_channels=[64, 128, 256, 256, 64], 
    metadata=data.metadata(), 
    edge_types=edge_types,
    rev_edge_types=rev_edge_types,
    # aggr_scheme='mean',
)

In [13]:
class GraphDataset(Dataset):
    def __init__(
        self,
        data,
        edge_types,
    ):
        self.data = data
        self.edge_types = edge_types

    def __len__(self):
        return len(self.edge_types)

    def __getitem__(self, idx):
        return self.data, self.edge_types[idx]
    
def collate_fn(input):
    data, edge_types = zip(*input)
    return data[0], edge_types

In [14]:
# for edge_types, rev_edge_types in edges:
log_dir = 'results/log/cd/gatv2'
loss_checkpoint_dir = 'results/checkpoints/cd/gatv2/loss'
auc_checkpoint_dir = 'results/checkpoints/cd/gatv2/silhouette_coefficient'

os.makedirs(log_dir, exist_ok=True)
os.makedirs(loss_checkpoint_dir, exist_ok=True)
os.makedirs(auc_checkpoint_dir, exist_ok=True)


lr = 1e-3
optim = torch.optim.Adam(model.parameters(), lr=lr)
model.set_optimizer(optim)

wandb_logger = WandbLogger(
    project="CommunityDectection_finalv1_gatv2",
    log_model=True,
    save_dir=log_dir,
    name=f"ALL",
    entity='ssc_project'

)

loss_checkpoint_callback = ModelCheckpoint(
    monitor=f'val_loss',
    dirpath=loss_checkpoint_dir,
    filename='ComDect-{epoch:02d}-{val_loss:.2f}',
    save_top_k=3,
    save_last=True,
    mode='min',
    every_n_epochs=1
)
roc_auc_checkpoint_callback = ModelCheckpoint(
    monitor=f'silhouette_coefficient',
    dirpath=auc_checkpoint_dir,
    filename='ComDect-{epoch:02d}-{silhouette_coefficient:.2f}',
    save_top_k=3,
    save_last=True,
    mode='max',
    every_n_epochs=1
)

trainer = L.Trainer(
    max_epochs=500,
    check_val_every_n_epoch=10,
    callbacks=[
        loss_checkpoint_callback, 
        roc_auc_checkpoint_callback,
    ],
    logger=wandb_logger,
    log_every_n_steps=1
)

train_loader = DataLoader(
    GraphDataset(train_data, edge_types),
    batch_size=2,
    shuffle=True,
    drop_last=False,
    pin_memory=True,
    num_workers=4,
    collate_fn=collate_fn,
)
val_loader = DataLoader(
    GraphDataset(val_data, edge_types),
    batch_size=2,
    shuffle=False,
    drop_last=False,
    pin_memory=True,
    num_workers=4,
    collate_fn=collate_fn
)

trainer.fit(model, train_loader, val_loader)
# wandb.finish()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
wandb: Currently logged in as: hontrn9122 (ssc_project). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.9/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:653: Checkpoint directory /notebooks/results/checkpoints/cd/gatv2/loss exists and is not empty.
/usr/local/lib/python3.9/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:653: Checkpoint directory /notebooks/results/checkpoints/cd/gatv2/silhouette_coefficient exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.9/dist-packages/lightning/pytorch/utilities/model_summary/model_summary.py:454: A layer with UninitializedParameter was found. Thus, the total number of parameters detected may be inaccurate.

  | Name    | Type              | Params
----------------------------------------------
0 | encoder | GraphModule       | 4.0 M 
1 | crit    | BCEWithLogitsLoss | 0     
----------------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
16.060    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

(2,)


OutOfMemoryError: CUDA out of memory. Tried to allocate 92.15 GiB. GPU 0 has a total capacity of 47.54 GiB of which 47.00 GiB is free. Process 820679 has 542.00 MiB memory in use. Of the allocated memory 137.56 MiB is allocated by PyTorch, and 88.44 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)